# Genearate Arctic Ocean Ecosystem boundaries conditions

In [1]:
import os
import sys
import numpy as np
from struct import *

### Functions

In [2]:
# dim = (nt,nZ,nX)
def read_bin(fname, dim):
    with open(fname, 'rb') as fid:
        tmp = np.fromfile(fid, '>f4')
    # Create a data array
    fld = tmp.reshape(dim) 
    return(fld)

In [3]:
def bin_save(fnm, array):
    newFile = open(fnm, "wb")
    bitearray = pack(">%sf" % len(array.flatten()),*array.flatten())
    newFile.write(bitearray);

### I. Set the reading and writing path

In [4]:
# Set parameters
HDlist = os.listdir('/Volumes') 
HDsel = np.nan
HD = ["Proteus","Itsasorantz"]
# HD Selection
while not (HDsel==1 or HDsel==2):
    # While loop Intilization
    if not np.isnan(HDsel):
        print("Wrong choice ! Type 1 or 2")
    # Test Both HD pluged in case    
    if HD[0] in HDlist and HD[1] in HDlist:
        HDsel = int(input("Do you want to read data on (1) Proteus or (2) Itsasorantz ?"))
    # Selection for 1 HD pluged in
    elif HD[0] in HDlist:
        HDsel = 1
    elif HD[1] in HDlist:
        HDsel = 2
print(f"Reading data on {HD[HDsel-1]}")

Reading data on Itsasorantz


In [5]:
HDpath = f"/Volumes/{HD[HDsel-1]}/"
In_pth = HDpath+"Modelling/Forcing/run_template/Mac270_Carroll/32_Tracers_Nr46/"
Out_pth = HDpath+"Modelling/Forcing/run_template/Mac270_Carroll/33_Tracers_Nr46/"

## II. Read original Open Boudaries files

### a. Set parameters

###### grid dimensions

In [6]:
## Time
nt = 12*29+1
## Depth
nZ = 46
## Width
nXw = 68
nXs = 46

###### Tracers list

In [7]:
# Initial Tracers list
InitTr_ls = ['DIC','NO3','NO2','NH4','PO4','Fet','SiO2','DOC','rDOC','DON','DOP','DOFe','POC','PON','POP','POFe',
             'POSi','PIC','ALK','O2','c1','c2','c3','c4','c5','c6','c7','chl1','chl2','chl3','chl4','chl5']
# Final Tracers list
FinaTr_ls = ['DIC','NO3','NO2','NH4','PO4','Fet','SiO2','DOC','rDOC','DON','DOP','DOFe','POC','PON','POP','POFe',
             'POSi','PIC','ALK','O2','CDOM','c1','c2','c3','c4','c5','c6','c7','chl1','chl2','chl3','chl4','chl5']

### b. Read data

###### Western Boundary

In [8]:
Init_OBW = []
for i in range(1,len(InitTr_ls)+1):
        Init_OBW.append(read_bin(In_pth+f"OBWtr{i}_Mac_46x68.bin", (nt,nZ,nXw)))

###### Southern Boundary

In [9]:
Init_OBS = []
for i in range(1,len(InitTr_ls)+1):
        Init_OBS.append(read_bin(In_pth+f"OBStr{i}_Mac_46x68.bin", (nt,nZ,nXs)))

## III. Create New tracers matrixes

In [10]:
Fina_OBW = Init_OBW.copy()
Fina_OBS = Init_OBS.copy()

### a. tDOC (tDOC backgroud is set constant to 50 $mmolC.m^{-3}$ whole water column)

###### Western Boundary

In [11]:
## Create the tDOC matrix
#tDOCw = np.ones((nt,nZ,nXw))*50
#tDOCw[Init_OBW[0]==0] = 0
## Add it into the new list
#Fina_OBW.insert(FinaTr_ls.index('tDOC'),tDOCw)

###### Eastern Boundary

In [12]:
## Create the tDOC matrix
#tDOCs = np.ones((nt,nZ,nXs))*50
#tDOCs[Init_OBS[0]==0] = 0
## Add it into the new list
#Fina_OBS.insert(FinaTr_ls.index('tDOC'),tDOCs)

### b. Disable specific plantkon types + Chl-a asociated

Create a list with tracers to disable

In [13]:
#TrcDis_ls = np.array([23,24,25,30,31,32])

Set 0 fields

In [14]:
#zero_fieldS = np.zeros(Fina_OBS[0].shape)
#zero_fieldW = np.zeros(Fina_OBW[0].shape)

In [15]:
#for i in TrcDis_ls-1:
#    # Southern boundary
#    Fina_OBS[i] = zero_fieldS
#    # Western boundary
#    Fina_OBW[i] = zero_fieldW

### c. Add CDOM (Set with DOC bondary field)  

In [16]:
# Add CDOM matrix Western boundary
Fina_OBW = np.insert(Fina_OBW, FinaTr_ls.index('CDOM'), 0.05*Init_OBW[InitTr_ls.index('DOC')], axis=0)
# Add CDOM matrix Western boundary
Fina_OBS = np.insert(Fina_OBS, FinaTr_ls.index('CDOM'), 0.05*Init_OBS[InitTr_ls.index('DOC')], axis=0)

## IV. Save the OBC files

In [17]:
svc = input("Do you want to save the New Open Boundary files created ? ('y' or 'n')\n")
if svc == 'y':
    print("Saving procedure allowed")
elif svc == 'n':
    print("Warning: files will not be saved")

Do you want to save the New Open Boundary files created ? ('y' or 'n')
y
Saving procedure allowed


In [18]:
if svc == 'y':
    for i in range(len(FinaTr_ls)):
        bin_save(Out_pth+f"OBWtr{i+1}_Mac_46x68.bin", Fina_OBW[i])
        bin_save(Out_pth+f"OBStr{i+1}_Mac_46x68.bin", Fina_OBS[i])